### Import all the necessary libraries

In [1]:
###### comment these lines if you execute code in local
try:
  import pennylane as qml
except ModuleNotFoundError:
  !pip install pennylane
  import pennylane as qml
######

import pennylane as qml
from pennylane import numpy as np
from pennylane.templates.embeddings import AngleEmbedding
from pennylane.templates.layers import StronglyEntanglingLayers
from pennylane.optimize import GradientDescentOptimizer
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
# %matplotlib inline
import torch
from torch.autograd import Function, Variable
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.impute import SimpleImputer
import random
import math


np.random.seed = 7
torch.manual_seed(7)
random.seed(7)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 38.7 MB/s eta 0:00:00


### Data generation and processing

In [2]:
print('Data of Solar Radiation on specific points of Earth\'s surface\nEach row is a point, each column is a value of solar radiation in time, sampled each 30 min for a month')

data = pd.read_csv('data.csv').iloc[0].values
print(f'shape of data: {data.shape}')

print('\nFor simplicity, take only the first 8 days\n')
data = data[:8*24*2]
print(f'shape of data: {data.shape}')

print(f'number of missing data: {len(data[data!=data])}')
print(f'index of missing data: {np.asarray(data!=data).nonzero()}')

plt.plot(data)
plt.show()

Data of Solar Radiation on specific points of Earth's surface
Each row is a point, each column is a value of solar radiation in time, sampled each 30 min for a month


FileNotFoundError: ignored

In [ ]:
def handle_missing_data(dataset,i):
    if i == 0:
        res = dataset[i+48]
    elif i== len(dataset)-1:
        res = dataset[i-48]
    else:
        res = (dataset[i-1]+dataset[i+1])/2
    return res

for i in range(len(data)):
    if data[i]!=data[i]:
        data[i] = handle_missing_data(data,i)

plt.plot(data)
plt.show()

In [ ]:
len_train = 0.8
window_size = 5
prediction_distance = 1
train = data[:int(len_train*len(data))].reshape(-1,1)
test = data[int(len_train*len(data))-window_size:].reshape(-1,1)

scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

def split_sequence(sequence, n_steps, prediction_distance):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix + prediction_distance-1> len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix+prediction_distance-1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

trainX, trainY = split_sequence(train, window_size ,prediction_distance)
testX, testY = split_sequence(test, window_size, prediction_distance)

In [ ]:
plt.plot(train)
plt.title('Train')
plt.show()

In [ ]:
plt.plot(test)
plt.title('Test')
plt.show()

In [ ]:

trainX = Variable(torch.Tensor(np.array(trainX))).view(-1,window_size,1)
trainY = Variable(torch.Tensor(np.array(trainY))).view(-1,1,1)

testX = Variable(torch.Tensor(np.array(testX))).view(-1,window_size,1)
testY = Variable(torch.Tensor(np.array(testY))).view(-1,1,1)


In [ ]:
from torch.utils.data import Dataset
class TimeseriesDataset(Dataset):   
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, index):
        return [self.X[index,:,:], self.y[index,:]]


In [ ]:
train_dataset = TimeseriesDataset(trainX, trainY)
test_dataset = TimeseriesDataset(testX, testY)

In [ ]:
batch_size = 8 #len(train_dataset)

train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(dataset = test_dataset, batch_size = len(test_dataset))

### QGRU

In [ ]:
class QGRU(nn.Module):
    def __init__(self, 
                input_size=1, 
                hidden_size=3, 
                n_qubits=4,
                n_qlayers=2,
                batch_first=True,
                return_sequences=False, 
                return_state=False,
                backend="lightning.qubit"):
        super(QGRU, self).__init__()
        self.n_inputs = input_size
        self.hidden_size = hidden_size
        self.concat_size = self.n_inputs + self.hidden_size
        self.n_qubits = n_qubits
        self.n_qlayers = n_qlayers
        self.backend = backend  # "default.qubit", "qiskit.basicaer", "qiskit.ibm"
        self.fc1 = nn.Linear(self.hidden_size, 1)

        self.batch_first = batch_first
        self.return_sequences = return_sequences
        self.return_state = return_state

        self.wires_reset = [f"wire_reset_{i}" for i in range(self.n_qubits)]
        self.wires_update = [f"wire_update_{i}" for i in range(self.n_qubits)]
        self.wires_output = [f"wire_output_{i}" for i in range(self.n_qubits)]

        self.dev_reset = qml.device(self.backend, wires=self.wires_reset)
        self.dev_update = qml.device(self.backend, wires=self.wires_update)
        self.dev_output = qml.device(self.backend, wires=self.wires_output)

        def _circuit_reset(inputs, weights):
            qml.templates.AngleEmbedding(inputs, wires=self.wires_reset)
            qml.templates.BasicEntanglerLayers(weights, wires=self.wires_reset)
            return [qml.expval(qml.PauliZ(wires=w)) for w in self.wires_reset]
        self.qlayer_reset = qml.QNode(_circuit_reset, self.dev_reset, interface="torch", diff_method="adjoint")

        def _circuit_update(inputs, weights):
            qml.templates.AngleEmbedding(inputs, wires=self.wires_update)
            qml.templates.BasicEntanglerLayers(weights, wires=self.wires_update)
            return [qml.expval(qml.PauliZ(wires=w)) for w in self.wires_update]
        self.qlayer_update = qml.QNode(_circuit_update, self.dev_update, interface="torch", diff_method="adjoint")

        def _circuit_output(inputs, weights):
            qml.templates.AngleEmbedding(inputs, wires=self.wires_output)
            qml.templates.BasicEntanglerLayers(weights, wires=self.wires_output)
            return [qml.expval(qml.PauliZ(wires=w)) for w in self.wires_output]
        self.qlayer_output = qml.QNode(_circuit_output, self.dev_output, interface="torch", diff_method="adjoint")

        weight_shapes = {"weights": (n_qlayers, n_qubits)}
        print(f"weight_shapes (n_qlayers, n_qubits) = ({n_qlayers}, {n_qubits})")

        #Define QGRU layers
        self.clayer_in = nn.Linear(self.concat_size, n_qubits)

        self.vqc_reset =  qml.qnn.TorchLayer(self.qlayer_reset, weight_shapes)
        self.vqc_update =  qml.qnn.TorchLayer(self.qlayer_update, weight_shapes)
        self.vqc_output =  qml.qnn.TorchLayer(self.qlayer_output, weight_shapes)

        self.clayer_out = nn.Linear(self.n_qubits, self.hidden_size)

    def forward(self, x, init_states=None):

        if self.batch_first is True:
            batch_size, seq_length, features_size = x.size()
        else:
            seq_length, batch_size, features_size = x.size()

        hidden_seq = []
        if init_states is None:
            h_t = torch.zeros(batch_size, self.hidden_size)  # hidden state (output)
        else:
            # for now we ignore the fact that in PyTorch you can stack multiple RNNs
            # so we take only the first elements of the init_states tuple init_states[0][0], init_states[1][0]
            h_t = init_states
            h_t = h_t[0]

        for t in range(seq_length):
            # get features from the t-th element in seq, for all entries in the batch
            x_t = x[:, t, :]

            # Concatenate input and hidden state
            v_t = torch.cat((h_t, x_t), dim=1)

            # match qubit dimension
            y_t = self.clayer_in(v_t)

            r_t = torch.sigmoid(self.clayer_out(self.vqc_reset(y_t)))  # forget block
            z_t = torch.sigmoid(self.clayer_out(self.vqc_update(y_t)))  # update block

            # Concatenate input and hidden state
            v2_t = torch.cat(((r_t * h_t), x_t), dim=1)

            # match qubit dimension
            y2_t = self.clayer_in(v2_t)
            
            h_tilde_t = torch.tanh(self.clayer_out(self.vqc_output(y2_t)))

            h_t = ((1-z_t) * h_tilde_t) + (z_t * h_t)

            hidden_seq.append(h_t.unsqueeze(0))
        hidden_seq = torch.cat(hidden_seq, dim=0)
        hidden_seq = hidden_seq.transpose(0, 1).contiguous()
        x = self.fc1(h_t)
        return x

### LSTM

In [ ]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.hidden1 = 5
        self.lstm = nn.LSTM(1,self.hidden1,1,batch_first=True)
        self.fc1 = nn.Linear(self.hidden1, 1)
            
    def forward(self, x):        
        _, (x,_) = self.lstm(x)
        x = x.view(-1, self.hidden1)
        x = self.fc1(x)
        return x
    
    
    def predict(self, x):
        pred = self.forward(x)
        return torch.tensor(pred)

### Training

In [ ]:
runs=1

# number of epochs to train the model
epochs = 100

# arrays to save evaluation results
final_MSE = np.zeros((epochs,runs))
final_MAE = np.zeros((epochs,runs))

for i in range(0,runs):
    print('Run:',i)
    rng=np.random.default_rng(i)
    torch.manual_seed(i)
    random.seed(i)
    
    network = QGRU()
    
    # specify optimizer (RMSprop) and learning rate
    optimizer = optim.RMSprop(network.parameters(), lr=0.01)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=95, gamma=0.7) 

    # specify loss function (Mean Squared Error)
    loss_func = nn.MSELoss()
    
    epoch_train_losses = []
    epoch_test_losses = []
    for epoch in range(0,epochs):

      # monitor training loss
      train_loss = 0.0
      test_loss = 0.0

      ###################
      # train the model #
      ###################
      network.train() # prep model for training
      for batch_idx, (x,y) in enumerate(train_loader):
        # clear the gradients of all optimized variables
        optimizer.zero_grad(set_to_none=True)   
        # forward pass: compute predicted outputs by passing inputs to the model
        output = network(x).reshape(-1,1)
        # calculate the loss (MSELoss automatically computes the mean of the sum of the batch losses)
        loss = (loss_func(output, y.view(-1,1)))
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*x.size(0)

        if batch_idx%10 == 0:
            print('\rEpoch %d ~ Batch %d (of %d) ~ Loss %f ' % (epoch, batch_idx, len(train_loader)-1, loss.item()), end='\n')
  
      ######################    
      # validate the model #
      ######################
      network.eval() # prep model for evaluation
      with torch.no_grad():
        for x,y in test_loader:
          # forward pass: compute predicted outputs by passing inputs to the model
          output = network(x).reshape(-1,1)

          # calculate the loss
          loss = (loss_func(output, y.view(-1,1)))
          # update running validation loss 
          test_loss += loss.item()*x.size(0)

      # print training/validation statistics 
      # calculate average loss over an epoch
      train_loss = train_loss/len(train_loader.sampler)
      test_loss = test_loss/len(test_loader.sampler)
      epoch_train_losses.append(train_loss)
      epoch_test_losses.append(test_loss)

      print('\nEpoch %d ~ Loss Training %f ~ Loss Test %f' % (epoch, np.mean(train_loss), np.mean(test_loss)), end='\n\n')
      scheduler.step()

      predictions = network(testX).reshape(-1,1).cpu().detach().numpy()
      testScore = (mean_squared_error(scaler.inverse_transform(testY.view(-1,1)), scaler.inverse_transform(predictions)))
      final_MSE[epoch,i] = testScore
      if epoch % 5 == 4:
        print('Test Score: %.5f MSE' % (testScore))
        print('epoch:',epoch)

      testScore_MAE = mean_absolute_error(scaler.inverse_transform(testY.view(-1,1)), scaler.inverse_transform(predictions))
      final_MAE[epoch,i] = testScore_MAE

    print('Test Score: %.5f MSE' % (testScore))
    print('Test Score: %.5f MAE\n' % (testScore_MAE))

In [ ]:
print('Test Score: %.5f MSE' % (testScore))
print('Test Score: %.5f MAE\n' % (testScore_MAE))

# Errors and Plots

In [ ]:
plt.plot(epoch_train_losses)
plt.plot(epoch_test_losses)
plt.title('NN Training Convergence for {} epochs'.format(epochs))
plt.xlabel('Epochs')
plt.ylabel('MSE Loss')
plt.legend(['Training', 'Test'])
plt.show()

In [ ]:
predictions = scaler.inverse_transform(np.reshape(predictions,(-1,1)))
testY = scaler.inverse_transform(testY.view(-1,1))
plt.plot(predictions)
plt.plot(testY)
plt.title('NN Test Predictions vs Ground Truth')
plt.xlabel('x')
plt.ylabel('Surface Solar Radiation (W/m^2)')
plt.legend(['Predictions', 'Ground truth'])
plt.show()